In [14]:
import os
import time
import math

import numpy as np
import tensorflow as tf
import cv2

slim = tf.contrib.slim

In [2]:
import sys
sys.path.append('../')

In [3]:
# isess = tf.InteractiveSession()

In [4]:
%matplotlib inline

# CNNs benchmarking

Fairly simple benchmarks. Absolute numbers are probably rubbish, but should give a good indication of relative performance between architectures.

In [5]:
from models_slim import mobilenets, inception_v1, inception_v2, inception_v3


In [6]:
# Main parameters.
inshape = [16, 289, 289, 3]

In [43]:
# MobileNets.
def build_mobilenets(inshape, data_format='NHWC'):
    tf.reset_default_graph()
    with slim.arg_scope(mobilenets.mobilenets_arg_scope(data_format=data_format, is_training=False)):
        inputs = tf.placeholder(tf.float32, shape=inshape)
        outputs = mobilenets.mobilenets(inputs, is_training=False)
        return inputs, [outputs]
    
# Inception v1, v2, v3, v4, ...
def build_inception_v1(inshape, data_format='NHWC'):
    tf.reset_default_graph()
    with slim.arg_scope(inception_v1.inception_v1_arg_scope(data_format=data_format, is_training=False)):
        inputs = tf.placeholder(tf.float32, shape=inshape)
        outputs = inception_v1.inception_v1(inputs, is_training=False)
        return inputs, [outputs]

In [44]:
def benchmark_network(inputs, outputs, nruns=100):
    """Benchmark a network.
    """
    inshape = inputs.get_shape().as_list()
    with tf.Session() as sess:
        # Initialize variables.
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        # Run some loops...
        runs = []
        for i in range(nruns):
            start = time.time()
            sess.run(outputs, feed_dict={inputs: np.random.random(inshape)})
            end = time.time()
            runs.append(end - start)
        print('Average: %.5f sec.' % np.mean(runs))
        print('Standard deviation: %.5f sec.' % np.std(runs))
            

In [48]:
inputs, outputs = build_mobilenets(inshape, data_format='NHWC')
benchmark_network(inputs, outputs, nruns=4)

Average: 3.32509 sec.
Standard deviation: 0.72775 sec.


In [49]:
inputs, outputs = build_inception_v1(inshape, data_format='NHWC')
benchmark_network(inputs, outputs, nruns=4)

ValueError: Can not squeeze dim[1], expected a dimension of 1, got 4 for 'InceptionV1/Logits/SpatialSqueeze/Squeeze' (op: 'Squeeze') with input shapes: [16,4,4,1000].

1497548282.0432737

array([[ 0.9786032 ,  0.93100166],
       [ 0.36303134,  0.1462775 ]])